In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

# Carregar o modelo e o tokenizador
model_name_or_path = "TheBloke/zephyr-7B-beta-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")

# Criar o pipeline para geração de texto
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
)

# Definir o LLM usando o pipeline
llm = HuggingFacePipeline(pipeline=pipe)

# Criar um template de prompt
template = """
You are a helpful assistant that provides useful information and engages in casual conversation.
Respond naturally to user queries and provide useful information.
Please, write a single reply only!

Current conversation:
{history}
Question: {input}
"""

# Definir o PromptTemplate
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

# Definir a memória para a conversa
memory = ConversationBufferWindowMemory(k=3)

# Criar a ConversationChain
conversation_chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

# Definir a função get_response
def get_response(input_text):
    return conversation_chain.predict(input=input_text)

# Teste a função
input_question = "How are you?"
response = get_response(input_question)
print(response)

#### Caso dê um bug na saída, instale este AutoGPTQ Transformers

In [ ]:
!pip install auto-gptq transformers

#### Caso não esteja instalado a biblioteca do AutoGPTQ

In [ ]:
!pip install auto-gptq

#### Provavelmente exigirá esta instalação

In [ ]:
!pip install langchain_community

#### Esta bibilioteca faz parte para o funcionamento do Chatbot (Após instalar reinicie a sessão para entrar em vigor)

In [ ]:
!pip install optimum

#### Caso dê algum erro na compilação, instale esta biblioteca extra

In [ ]:
!pip install langchain langchain_community

######INSERIR O TEMPLATE EM PORTUGUÊS

In [ ]:
template_pt = """
Você é um assistente útil que fornece informações e participa de conversas do dia a dia.
Responda de forma simples, curta e natural, oferecendo informações úteis.
Por favor, escreva apenas uma resposta!

Conversa atual:
{history}
Pergunta: {input}
"""

### teste bot1 - pergunta

In [ ]:
input_question = "Qual é a capital do Brasil?"
response = get_response(input_question)
print(response)

#### teste bot2 - saída

In [ ]:
print(get_response("Olá!"))

In [ ]:
input_question = "E qual é a terceira cidade mais populosa?"

#### utilize para limpar o histórico das conversas anteriores e criar uma nova conversa ou interação

In [ ]:
memory.clear()  # limpa o histórico de conversas
print(get_response("Hoje o dia está como?!"))

#### configurando com outro template em português

In [ ]:
template = """
Você é um assistente útil que fornece informações úteis e conversa naturalmente em português.
Responda às perguntas de maneira clara e direta, apenas em português.
Por favor, escreva apenas uma resposta por vez.

Conversa atual:
{history}
Pergunta: {input}
"""

#### teste bot3 - verificar qual idioma que o chatbot está configurado

In [ ]:
print(get_response("Em que idioma você está respondendo?"))

#### configurando o template para responder "somente" em português

In [ ]:
template = """
Você é um assistente útil que fornece informações claras e objetivas.
Responda **somente** em português.
Não use outro idioma.

Conversa atual:
{history}
Pergunta: {input}
"""

#### teste bot4 - limpando o histórico e verificando em qual idioma está o chatbot

In [ ]:
memory.clear()  # Limpa o histórico para garantir que o ajuste tenha efeito
print(get_response("Em que idioma você está respondendo?"))

#### ajustes para o chatbot datas

In [ ]:
from datetime import datetime, date

# Dicionário com as datas dos feriados e eventos fixos no Brasil
feriados = {
    "Ano Novo": date(datetime.today().year, 1, 1),
    "Carnaval": date(datetime.today().year, 2, 13),  # Ajuste anual necessário
    "Páscoa": date(datetime.today().year, 3, 31),  # Ajuste anual necessário
    "Dia do Trabalho": date(datetime.today().year, 5, 1),
    "Dia dos Namorados": date(datetime.today().year, 6, 12),
    "Independência do Brasil": date(datetime.today().year, 9, 7),
    "Dia das Crianças": date(datetime.today().year, 10, 12),
    "Finados": date(datetime.today().year, 11, 2),
    "Proclamação da República": date(datetime.today().year, 11, 15),
    "Natal": date(datetime.today().year, 12, 25),
    "Dia dos Pais": None,  # Calculado dinamicamente (segundo domingo de agosto)
    "Dia das Mães": None   # Calculado dinamicamente (segundo domingo de maio)
}

# Função para calcular o segundo domingo de um mês
def segundo_domingo(mes, ano):
    primeiro_dia = date(ano, mes, 1)
    return primeiro_dia.replace(day=(8 - primeiro_dia.weekday()))

# Definir dinamicamente os dias dos pais e das mães
ano_atual = datetime.today().year
feriados["Dia dos Pais"] = segundo_domingo(8, ano_atual)
feriados["Dia das Mães"] = segundo_domingo(5, ano_atual)

# Função para calcular os dias restantes para um evento
def dias_ate_evento(evento):
    hoje = datetime.today().date()
    data_evento = feriados.get(evento)

    if data_evento:
        dias_faltando = (data_evento - hoje).days
        if dias_faltando < 0:
            # Se a data já passou neste ano, calcula para o próximo ano
            data_evento = data_evento.replace(year=ano_atual + 1)
            dias_faltando = (data_evento - hoje).days

        return f"Faltam {dias_faltando} dias para {evento}."
    else:
        return f"Data de {evento} não encontrada."

# Testes
print(dias_ate_evento("Dia dos Pais"))
print(dias_ate_evento("Natal"))
print(dias_ate_evento("Independência do Brasil"))
print(dias_ate_evento("Carnaval"))

#### limpando o histórico e testando as habilidades do chatbot

In [ ]:
memory.clear()  # Limpa o histórico para garantir que o ajuste tenha efeito
print(get_response("você é um assistente que precisará responder com informações úteis e conversas do dia a dia. Responda de forma simples, curta e natural com informações úteis.?"))

#### <!> Atenção <!>
##### Utilize no lugar da linha de comando "ngrok config add-authtoken "inserir o SEU TOKEN AQUI GERADO NO SITE" e se for disponilizar em seu repositório, remova o seu token e insira um texto indicando onde outro colaborador ou desenvolvedor irá colocar o token dele.

#### Projeto Completo - 7DaysOfCode

In [ ]:


# Atenção: se você quiser rodar mais de uma vez as células, esta aqui só é necessária uma vez no mesmo ambiente de execução
!pip3 install chainlit pyngrok langchain optimum auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ -q


!ngrok config add-authtoken "2tBX42Q7po6pD4usJ9lkFQ8CuZK_7oXGJvRGWYPeaYKa4s7f7"


from pyngrok import ngrok
print(ngrok.connect(8000).public_url)


%%writefile app.py
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
import chainlit as cl

model_name_or_path = "TheBloke/zephyr-7B-beta-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
    )

llm=HuggingFacePipeline(pipeline=pipe)

template = """

You are a helpful assistant that provides information and engages in casual conversation.
Respond naturally to user queries and provide useful information.
Please, write a single reply only!


Current conversation:
{history}
Question: {input}


"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)
memory=ConversationBufferWindowMemory(k=3)

@cl.on_chat_start
async def start():
    llm_chain = ConversationChain(prompt=prompt, llm=llm, memory=memory)
    cl.user_session.set("llm_chain", llm_chain)

@cl.on_message
async def main(message: cl.message):
    llm_chain = cl.user_session.get("llm_chain")
    cb = cl.AsyncLangchainCallbackHandler()
    cb.answer_reached = True

    res = await cl.make_async(llm_chain)(message.content, callbacks=[cb])

    response_text = res['response'].split("Answer:")[-1].strip()

    await cl.Message(content=response_text).send()


!chainlit run app.py

#### Extra: Em caso de erro da primeira compilação, utilize este abaixo

In [ ]:
# Instalação das dependências
!pip3 install chainlit pyngrok langchain optimum auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ -q

# Configuração do ngrok
!ngrok config add-authtoken "2tBX42Q7po6pD4usJ9lkFQ8CuZK_7oXGJvRGWYPeaYKa4s7f7"

# Conectando o ngrok
from pyngrok import ngrok
print(ngrok.connect(8000).public_url)

# Criando o arquivo app.py manualmente
app_py_content = """
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
import chainlit as cl

model_name_or_path = "TheBloke/zephyr-7B-beta-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
    )

llm=HuggingFacePipeline(pipeline=pipe)

template = \"""
You are a helpful assistant that provides information and engages in casual conversation.
Respond naturally to user queries and provide useful information.
Please, write a single reply only!

Current conversation:
{history}
Question: {input}
\"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)
memory=ConversationBufferWindowMemory(k=3)

@cl.on_chat_start
async def start():
    llm_chain = ConversationChain(prompt=prompt, llm=llm, memory=memory)
    cl.user_session.set("llm_chain", llm_chain)

@cl.on_message
async def main(message: cl.message):
    llm_chain = cl.user_session.get("llm_chain")
    cb = cl.AsyncLangchainCallbackHandler()
    cb.answer_reached = True

    res = await cl.make_async(llm_chain)(message.content, callbacks=[cb])

    response_text = res['response'].split("Answer:")[-1].strip()

    await cl.Message(content=response_text).send()
"""

# Salvando o conteúdo no arquivo app.py
with open("app.py", "w") as f:
    f.write(app_py_content)

# Executando o Chainlit
!chainlit run app.py -w

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
https://c573-34-141-163-103.ngrok-free.app
2025-02-20 21:14:45.775353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740086085.814810    3511 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740086085.834181    3511 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py:549: LangChainDeprecationWarning: Importing LLMs from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.llms import HuggingFacePipeline`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
tokenizer_config.json: 100% 1.43k/1.43k [00:00<00:00, 6.12MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 13.1MB/s]
tokenizer.json: 100% 1.80M/1.80M [00:00<00:00, 5.26MB/s]
special_tokens_map.json: 100% 168/168 [00:00<00:00, 912kB/s]
config.json: 100% 1.31k/1.31k [00:00<00:00, 8.11MB/s]
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
2025-02-20 21:14


Aborted!


#### Faça o cadastro no site ngrok para gerar o seu Token

#### <!> Importante <!>
#### ::::: SEGURANÇA:::::
##### Se disponibilizar em seu repositório este chatbot, para a sua segurança: "remova o seu token e insira o texto no lugar como SEU_TOKEN_AQUI para evitar problemas futuros ou o uso indevido por terceiros."

Lembre-se: O seu Token é único e exclusivo para o seu uso e ele é a sua identificação digital!

In [ ]:
# Atenção: se você quiser rodar mais de uma vez as células, esta aqui só é necessária uma vez no mesmo ambiente de execução
!pip3 install chainlit pyngrok langchain optimum auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ -q


!ngrok config add-authtoken "2tBX42Q7po6pD4usJ9lkFQ8CuZK_7oXGJvRGWYPeaYKa4s7f7" # Replace your_actual_authtoken with your real authtoken


from pyngrok import ngrok
print(ngrok.connect(8000).public_url)

#### Teste do Chatbot final

In [ ]:
!chainlit run app.py -w

2025-02-20 21:09:59 - Created default config file at /content/.chainlit/config.toml
2025-02-20 21:09:59 - Created default translation directory at /content/.chainlit/translations
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/ja.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/en-US.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/kn.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/gu.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/mr.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/hi.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/ta.json
2025-02-20 21:09:59 - Created default translation file at /content/.chainlit/translations/zh-CN.json
2025-02-20 21:09:59 - Created d

#### Se der erro, use este comando abaixo para verificar o Chainlit

In [ ]:
!pip show chainlit

#### Se estiver ausente, use este coma do para instalá-lo *** Ele irá reiniciar ou reinicie para que entre em vigor *** Logo, teste o chainlit novamente na célula '!chainlit run app.py -w'

In [ ]:
!pip install chainlit